<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/PlayingWithPandasStructures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing datastructures, exploring what's possible

# Setup env


In [0]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Google drive access

In [2]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 21
-rw------- 1 root root 16580 May 25 20:01 data_prep.py
drwx------ 2 root root  4096 May 25 16:59 __pycache__


## Auto reload module

Now using library code I've created and saved to google drive which is automatically pushed to the cloud and made available to the colab env. The autoreload stuff below should help imports to 'reimport' to load changes to the library code.

It's not the quickest/ most reliable, so if in a hurry, brute force loading of changes by restarting the runtime.

In [0]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

## Imports and accessing lib functions

In [5]:
# install required libs
!pip install mido

In [6]:
# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep

LOADING - data_prep.py module name is: data_prep


In [0]:
# imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np


# object that provides colours for charts
from itertools import cycle



In [8]:
# testing auto reload of modules 
data_prep.test_function_call('bling')

test function called worked! :)  bling


## Pandas display options

In [0]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 2000
    display.max_colwidth = 1000
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()
#pd.reset_option('all')


# Setup test file

## Load file

In [0]:
gmt = data_prep.GrooveMidiTools()

In [11]:
file_name = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/1_funk-groove1_138_beat_4-4.mid'
midi_file = data_prep.MIDI_File_Wrapper(file_name, gmt.mappings)
f = midi_file
f_df = f.df_midi_data

FILE: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/1_funk-groove1_138_beat_4-4.mid
    tracks: [<midi track 'Base Midi' 1037 messages>]
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=434783 time=0>
    note_on span - first tick: 5 , last tick: 30634 
    good instruments: 4, {36.0: 'Bass Drum 1 (36)', 38.0: 'Acoustic Snare (38)', 42.0: 'Closed Hi Hat (42)', 51.0: 'Ride Cymbal 1 (51)'}


... the above verifies I'm able to create custom objects from custom code, great!


## Setup MIDI event timing bins...

In [12]:
# MTT object for parsing file and
# calculating crticial time metrics
mtt = data_prep.MidiTimingTools(file_name, f.ticks(), f.tempo_us(), f.ts_num(), f.ts_denom(), f.last_hit())

# values needed these for making MultiIndex later
quantize_level = mtt.bins_per_bar()
bars_in_file = mtt.bars_in_file()
tp_beat = mtt.ts_ticks_per_beat()
tp_bin = mtt.bin_size()

print('bar info - bars in file: {}, bar quantize level: {}'.format(bars_in_file, quantize_level))
print('tick info - ticks per time sig beat: {}, ticks per quantize bin: {}'.format(tp_beat, tp_bin))

# capture timing data from MidiTimingTools in df...
beats_col, offsets_col = mtt.get_offsets(f_df[f.cum_ticks_col])
f_df['beat_offset'] = offsets_col
f_df['beat_center'] = beats_col
f_df['file_beat_number'] = pd.Categorical(f_df.beat_center).codes

f_df.head(20)

bar info - bars in file: 16, bar quantize level: 16.0
tick info - ticks per time sig beat: 480.0, ticks per quantize bin: 120


,msg_type,delta_ticks,total_ticks,total_seconds,note,velocity,raw_data,beat_offset,beat_center,file_beat_number
0,track_name,0,0,0.000000,NaN,NaN,"{'type': 'track_name', 'name': 'Base Midi', 'time': 0}",0,0,0
1,instrument_name,0,0,0.000000,NaN,NaN,"{'type': 'instrument_name', 'name': 'Brooklyn', 'time': 0}",0,0,0
2,time_signature,0,0,0.000000,NaN,NaN,"{'type': 'time_signature', 'numerator': 4, 'denominator': 4, 'clocks_per_click': 24, 'notated_32nd_notes_per_beat': 8, 'time': 0}",0,0,0
3,key_signature,0,0,0.000000,NaN,NaN,"{'type': 'key_signature', 'key': 'C', 'time': 0}",0,0,0
4,smpte_offset,0,0,0.000000,NaN,NaN,"{'type': 'smpte_offset', 'frame_rate': 24, 'hours': 33, 'minutes': 1, 'seconds': 15, 'frames': 16, 'sub_frames': 24, 'time': 0}",0,0,0
5,set_tempo,0,0,0.000000,NaN,NaN,"{'type': 'set_tempo', 'tempo': 434783, 'time': 0}",0,0,0
6,control_change,4,4,0.003623,NaN,NaN,"{'type': 'control_change', 'time': 4, 'control': 4, 'value': 77, 'channel': 9}",4,0,0
7,note_on,1,5,0.004529,42.0,55.0,"{'type': 'note_on', 'time': 1, 'note': 44, 'velocity': 55, 'channel': 9}",5,0,0
8,note_on,4,9,0.008152,36.0,39.0,"{'type': 'note_on', 'time': 4, 'note': 36, 'velocity': 39, 'channel': 9}",9,0,0
9,note_on,6,15,0.013587,51.0,67.0,"{'type': 'note_on', 'time': 6, 'note': 51, 'velocity': 67, 'channel': 9}",15,0,0


In [13]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037 entries, 0 to 1036
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   msg_type          1037 non-null   string  
 1   delta_ticks       1037 non-null   int64   
 2   total_ticks       1037 non-null   int64   
 3   total_seconds     1037 non-null   float64 
 4   note              746 non-null    float64 
 5   velocity          746 non-null    float64 
 6   raw_data          1037 non-null   string  
 7   beat_offset       1037 non-null   int64   
 8   beat_center       1037 non-null   category
 9   file_beat_number  1037 non-null   int16   
dtypes: category(1), float64(3), int16(1), int64(3), string(2)
memory usage: 69.1 KB


## Setup columns indicating bars & beats

These new columns will be needed for the new MultiIndex that needs to be created

In [0]:
# make a copy, just to practice on 
tmp_df = f_df.copy(deep=True)

In [0]:
# add column for bear index
tmp_df['bar_number'] = (tmp_df.file_beat_number // quantize_level) + 1
tmp_df['bar_number'] = tmp_df['bar_number'].astype(int) 

In [0]:
# add column for beat within the bar index
tmp_df['bar_beat_number'] = (tmp_df.file_beat_number % 16) + 1

In [0]:
# filter to only note_on events
tmp_df = tmp_df[tmp_df['msg_type'] == 'note_on'].copy() 


In [18]:
tmp_df.info()
tmp_df.head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373 entries, 7 to 1034
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   msg_type          373 non-null    string  
 1   delta_ticks       373 non-null    int64   
 2   total_ticks       373 non-null    int64   
 3   total_seconds     373 non-null    float64 
 4   note              373 non-null    float64 
 5   velocity          373 non-null    float64 
 6   raw_data          373 non-null    string  
 7   beat_offset       373 non-null    int64   
 8   beat_center       373 non-null    category
 9   file_beat_number  373 non-null    int16   
 10  bar_number        373 non-null    int64   
 11  bar_beat_number   373 non-null    int16   
dtypes: category(1), float64(3), int16(2), int64(4), string(2)
memory usage: 31.5 KB


,msg_type,delta_ticks,total_ticks,total_seconds,note,velocity,raw_data,beat_offset,beat_center,file_beat_number,bar_number,bar_beat_number
7,note_on,1,5,0.004529,42.0,55.0,"{'type': 'note_on', 'time': 1, 'note': 44, 'velocity': 55, 'channel': 9}",5,0,0,1,1
8,note_on,4,9,0.008152,36.0,39.0,"{'type': 'note_on', 'time': 4, 'note': 36, 'velocity': 39, 'channel': 9}",9,0,0,1,1
9,note_on,6,15,0.013587,51.0,67.0,"{'type': 'note_on', 'time': 6, 'note': 51, 'velocity': 67, 'channel': 9}",15,0,0,1,1
14,note_on,100,226,0.204710,36.0,41.0,"{'type': 'note_on', 'time': 100, 'note': 36, 'velocity': 41, 'channel': 9}",-14,240,2,1,3
15,note_on,32,258,0.233696,51.0,58.0,"{'type': 'note_on', 'time': 32, 'note': 51, 'velocity': 58, 'channel': 9}",18,240,2,1,3
18,note_on,7,344,0.311594,36.0,6.0,"{'type': 'note_on', 'time': 7, 'note': 36, 'velocity': 6, 'channel': 9}",-16,360,3,1,4
23,note_on,0,478,0.432971,42.0,67.0,"{'type': 'note_on', 'time': 0, 'note': 44, 'velocity': 67, 'channel': 9}",-2,480,4,1,5
24,note_on,17,495,0.448370,51.0,119.0,"{'type': 'note_on', 'time': 17, 'note': 51, 'velocity': 119, 'channel': 9}",15,480,4,1,5
26,note_on,0,513,0.464674,38.0,106.0,"{'type': 'note_on', 'time': 0, 'note': 40, 'velocity': 106, 'channel': 9}",33,480,4,1,5
30,note_on,122,746,0.675725,51.0,45.0,"{'type': 'note_on', 'time': 122, 'note': 51, 'velocity': 45, 'channel': 9}",26,720,6,1,7


In [0]:
tmp_df.set_index(['bar_number', 'bar_beat_number'], inplace=True)

In [20]:
tmp_df.index.names

FrozenList(['bar_number', 'bar_beat_number'])

In [21]:
# select first bar
tmp_df.loc[1]

,msg_type,delta_ticks,total_ticks,total_seconds,note,velocity,raw_data,beat_offset,beat_center,file_beat_number
bar_beat_number,,,,,,,,,,
1,note_on,1,5,0.004529,42.0,55.0,"{'type': 'note_on', 'time': 1, 'note': 44, 'velocity': 55, 'channel': 9}",5,0,0
1,note_on,4,9,0.008152,36.0,39.0,"{'type': 'note_on', 'time': 4, 'note': 36, 'velocity': 39, 'channel': 9}",9,0,0
1,note_on,6,15,0.013587,51.0,67.0,"{'type': 'note_on', 'time': 6, 'note': 51, 'velocity': 67, 'channel': 9}",15,0,0
3,note_on,100,226,0.204710,36.0,41.0,"{'type': 'note_on', 'time': 100, 'note': 36, 'velocity': 41, 'channel': 9}",-14,240,2
3,note_on,32,258,0.233696,51.0,58.0,"{'type': 'note_on', 'time': 32, 'note': 51, 'velocity': 58, 'channel': 9}",18,240,2
4,note_on,7,344,0.311594,36.0,6.0,"{'type': 'note_on', 'time': 7, 'note': 36, 'velocity': 6, 'channel': 9}",-16,360,3
5,note_on,0,478,0.432971,42.0,67.0,"{'type': 'note_on', 'time': 0, 'note': 44, 'velocity': 67, 'channel': 9}",-2,480,4
5,note_on,17,495,0.448370,51.0,119.0,"{'type': 'note_on', 'time': 17, 'note': 51, 'velocity': 119, 'channel': 9}",15,480,4
5,note_on,0,513,0.464674,38.0,106.0,"{'type': 'note_on', 'time': 0, 'note': 40, 'velocity': 106, 'channel': 9}",33,480,4


In [22]:
# selects bars 3-4
tmp_df.loc[3:4]

msg_type  delta_ticks  total_ticks  total_seconds  \
bar_number bar_beat_number                                                     
3          1                note_on           24         3846       3.483699   
           1                note_on            3         3849       3.486416   
           1                note_on            0         3852       3.489134   
           2                note_on           13         3977       3.602358   
           3                note_on           96         4073       3.689315   
           3                note_on            2         4075       3.691127   
           4                note_on            4         4162       3.769931   
           5                note_on            0         4303       3.897648   
           5                note_on           34         4337       3.928446   
           5                note_on            8         4346       3.936598   
           7                note_on          122         4579       4.147649   
           8                note_on            4         4695       4.252721   
           9                note_on           14         4820       4.365946   
           9                note_on            0         4823       4.368663   
           10               note_on           14         4949       4.482794   
           11               note_on          100         5049       4.573374   
           11               note_on           10         5059       4.582432   
           12               note_on           53         5112       4.630439   
           12               note_on           24         5143       4.658519   
           12               note_on           10         5161       4.674823   
           12               note_on           11         5181       4.692939   
           13               note_on            0         5281       4.783519   
           13               note_on           25         5306       4.806164   
           15               note_on          122         5540       5.018120   
           15               note_on            5         5545       5.022649   
4          1                note_on          111         5767       5.223737   
           1                note_on            1         5781       5.236418   
           2                note_on           17         5874       5.320657   
           3                note_on            9         5994       5.429353   
           3                note_on           30         6024       5.456527   
           4                note_on           16         6121       5.544389   
           5                note_on            1         6197       5.613230   
           5                note_on            8         6240       5.652179   
           5                note_on            5         6245       5.656708   
           6                note_on           42         6351       5.752723   
           7                note_on           23         6486       5.875005   
           8                note_on           29         6595       5.973737   
           9                note_on            0         6722       6.088774   
           9                note_on            3         6725       6.091491   
           9                note_on            2         6727       6.093303   
           10               note_on            4         6842       6.197469   
           11               note_on            8         6961       6.305259   
           11               note_on           17         6978       6.320658   
           12               note_on           65         7043       6.379535   
           13               note_on            0         7194       6.516310   
           13               note_on            8         7202       6.523557   
           15               note_on          113         7426       6.726455   
           15               note_on            9         7435       6.734608   

                          

In [23]:
tmp_df.index.names

FrozenList(['bar_number', 'bar_beat_number'])

In [24]:
# extract all notes in the first 1/16th note of a bar
# NOTE: 'level' refers to index, level 0 = bar_number, level 1 = bar_beat_number
tmp_df[tmp_df.index.isin([1], level=1)]

msg_type  delta_ticks  total_ticks  total_seconds  \
bar_number bar_beat_number                                                     
1          1                note_on            1            5       0.004529   
           1                note_on            4            9       0.008152   
           1                note_on            6           15       0.013587   
2          1                note_on           16         1935       1.752719   
           1                note_on            3         1938       1.755436   
           1                note_on            1         1944       1.760871   
3          1                note_on           24         3846       3.483699   
           1                note_on            3         3849       3.486416   
           1                note_on            0         3852       3.489134   
4          1                note_on          111         5767       5.223737   
           1                note_on            1         5781       5.236418   
5          1                note_on            1         7649       6.928448   
           1                note_on            9         7658       6.936600   
           1                note_on            5         7663       6.941129   
6          1                note_on            6         9589       8.685696   
           1                note_on            2         9591       8.687508   
           1                note_on            1         9593       8.689319   
7          1                note_on            0        11516      10.431169   
           1                note_on            8        11524      10.438415   
           1                note_on            3        11527      10.441133   
8          1                note_on           31        13431      12.165772   
           1                note_on            1        13455      12.187511   
9          1                note_on            1        15353      13.906715   
           1                note_on           19        15372      13.923926   
           1                note_on            6        15384      13.934795   
10         1                note_on            0        17285      15.656717   
           1                note_on            2        17287      15.658529   
           1                note_on            1        17288      15.659434   
11         1                note_on            1        19210      17.400378   
           1                note_on            0        19210      17.400378   
           1                note_on            9        19220      17.409436   
12         1                note_on          124        21152      19.159438   
           1                note_on            0        21168      19.173930   
13         1                note_on            1        23012      20.844222   
           1                note_on            7        23019      20.850562   
           1                note_on            2        23021      20.852374   
14         1                note_on            0        24967      22.615057   
           1                note_on           10        24977      22.624115   
           1                note_on            1        24978      22.625020   
15         1                note_on            7        26894      24.360529   
           1                note_on            0        26897      24.363247   
           1                note_on            6        26903      24.368681   
16         1                note_on            1        28808      26.094226   
           1                note_on            3        28811      26.096944   
           1                note_on            8        28819      26.104190   

                            note  velocity  \
bar_number bar_beat_number                   
1          1                42.0      55.0   
           1                36.0      39.0   
           1                51.0      67.0   
2          1                51.0    

In [25]:
# extract all notes in the first 1/16th note of a bar
# NOTE: 'level' refers to index, level 0 = bar_number, level 1 = bar_beat_number

# all I've changed from the last cell is the level, this now
# selects every note played in the first bar, and is actually 
# a bit verbose, as same can be managed with -> tmp_df.loc[1]
tmp_df[tmp_df.index.isin([1], level=0)]

msg_type  delta_ticks  total_ticks  total_seconds  \
bar_number bar_beat_number                                                     
1          1                note_on            1            5       0.004529   
           1                note_on            4            9       0.008152   
           1                note_on            6           15       0.013587   
           3                note_on          100          226       0.204710   
           3                note_on           32          258       0.233696   
           4                note_on            7          344       0.311594   
           5                note_on            0          478       0.432971   
           5                note_on           17          495       0.448370   
           5                note_on            0          513       0.464674   
           7                note_on          122          746       0.675725   
           8                note_on            6          863       0.781704   
           9                note_on            1          991       0.897646   
           9                note_on            5          996       0.902175   
           10               note_on            8         1116       1.010870   
           11               note_on           40         1195       1.082429   
           11               note_on            8         1236       1.119566   
           12               note_on            8         1314       1.190218   
           13               note_on            1         1460       1.322465   
           13               note_on            5         1465       1.326994   
           13               note_on            3         1468       1.329711   
           15               note_on          122         1702       1.541668   
           16               note_on           30         1808       1.637683   

                            note  velocity  \
bar_number bar_beat_number                   
1          1                42.0      55.0   
           1                36.0      39.0   
           1                51.0      67.0   
           3                36.0      41.0   
           3                51.0      58.0   
           4                36.0       6.0   
           5                42.0      67.0   
           5                51.0     119.0   
           5                38.0     106.0   
           7                51.0      45.0   
           8                38.0      50.0   
           9                42.0      67.0   
           9                51.0      53.0   
           10               38.0      55.0   
           11               36.0      15.0   
           11               51.0      59.0   
           12               36.0      34.0   
           13               42.0      67.0   
           13               51.0      96.0   
           13               38.0     103.0   
           15               51.0      38.0   
           16               38.0      56.0   

                                                                                              raw_data  \
bar_number bar_beat_number                                                                               
1          1                  {'type': 'note_on', 'time': 1, 'note': 44, 'velocity': 55, 'channel': 9}   
           1                  {'type': 'note_on', 'time': 4, 'note': 36, 'velocity': 39, 'channel': 9}   
           1                  {'type': 'note_on', 'time': 6, 'note': 51, 'velocity': 67, 'channel': 9}   
           3                {'type': 'note_on', 'time': 100, 'note': 36, 'velocity': 41, 'channel': 9}   
           3                 {'type': 'note_on', 'time': 32, 'note': 51, 'velocity': 58, 'channel': 9}   
           4                   {'type': 'note_on', 'time': 7, 'note': 36, 'velocity': 6, 'channel': 9}   
           5                  {'type': 'note_on', 'time': 0, 'note': 44, 'velocity': 67, 'channel': 9}   
           5                {'type': 'note_on',

In [26]:
# extract all notes in some 1/16th note of a bar
# NOTE: 'level' refers to index, level 0 = bar_number, level 1 = bar_beat_number

# this will select all notes played in 6th or 16th note
# of every bar in the MIDI file ...
tmp_df[tmp_df.index.isin([6, 16], level=1)]

,,msg_type,delta_ticks,total_ticks,total_seconds,note,velocity,raw_data,beat_offset,beat_center,file_beat_number
bar_number,bar_beat_number,,,,,,,,,,
1,16,note_on,30,1808,1.637683,38.0,56.0,"{'type': 'note_on', 'time': 30, 'note': 38, 'velocity': 56, 'channel': 9}",8,1800,15
2,16,note_on,29,3711,3.361416,38.0,38.0,"{'type': 'note_on', 'time': 29, 'note': 38, 'velocity': 38, 'channel': 9}",-9,3720,31
4,6,note_on,42,6351,5.752723,38.0,39.0,"{'type': 'note_on', 'time': 42, 'note': 38, 'velocity': 39, 'channel': 9}",-9,6360,53
5,16,note_on,5,9472,8.579718,38.0,51.0,"{'type': 'note_on', 'time': 5, 'note': 38, 'velocity': 51, 'channel': 9}",-8,9480,79
6,16,note_on,13,11391,10.317944,38.0,53.0,"{'type': 'note_on', 'time': 13, 'note': 38, 'velocity': 53, 'channel': 9}",-9,11400,95
7,16,note_on,5,13288,12.036243,38.0,28.0,"{'type': 'note_on', 'time': 5, 'note': 38, 'velocity': 28, 'channel': 9}",-32,13320,111
8,16,note_on,12,15266,13.827911,38.0,85.0,"{'type': 'note_on', 'time': 12, 'note': 38, 'velocity': 85, 'channel': 9}",26,15240,127
9,16,note_on,18,17167,15.549833,38.0,46.0,"{'type': 'note_on', 'time': 18, 'note': 38, 'velocity': 46, 'channel': 9}",7,17160,143
10,16,note_on,25,19100,17.300740,38.0,48.0,"{'type': 'note_on', 'time': 25, 'note': 38, 'velocity': 48, 'channel': 9}",20,19080,159


In [27]:
# alternatively, this will select every note played
# in the 6th and 16th bars of the MIDI file ...
tmp_df.loc[tmp_df.index.isin([6, 16], level=0)]

msg_type  delta_ticks  total_ticks  total_seconds  \
bar_number bar_beat_number                                                     
6          1                note_on            6         9589       8.685696   
           1                note_on            2         9591       8.687508   
           1                note_on            1         9593       8.689319   
           2                note_on           20         9724       8.807979   
           3                note_on          104         9828       8.902182   
           3                note_on            6         9834       8.907617   
           5                note_on            1        10063       9.115044   
           5                note_on           19        10082       9.132255   
           5                note_on            8        10090       9.139501   
           7                note_on          122        10324       9.351458   
           8                note_on           10        10445       9.461059   
           9                note_on           21        10577       9.580625   
           9                note_on            0        10579       9.582436   
           10               note_on            3        10694       9.686603   
           11               note_on           75        10785       9.769031   
           11               note_on            6        10791       9.774465   
           11               note_on           24        10829       9.808886   
           12               note_on           18        10886       9.860516   
           13               note_on            0        11031       9.991857   
           13               note_on           10        11041      10.000915   
           13               note_on            0        11041      10.000915   
           15               note_on          138        11291      10.227364   
           16               note_on           13        11391      10.317944   
16         1                note_on            1        28808      26.094226   
           1                note_on            3        28811      26.096944   
           1                note_on            8        28819      26.104190   
           2                note_on           28        28958      26.230096   
           3                note_on           57        29015      26.281727   
           3                note_on           17        29032      26.297125   
           3                note_on           27        29059      26.321582   
           5                note_on            1        29281      26.522669   
           5                note_on            0        29315      26.553466   
           7                note_on          111        29538      26.755459   
           7                note_on            4        29542      26.759082   
           9                note_on          128        29781      26.975568   
           9                note_on            1        29799      26.991872   
           10               note_on            0        29877      27.062524   
           11               note_on           13        30001      27.174843   
           11               note_on            3        30004      27.177561   
           11               note_on           22        30026      27.197488   
           12               note_on            6        30121      27.283539   
           13               note_on            0        30230      27.382271   
           13               note_on           23        30256      27.405822   
           13               note_on            7        30263      27.412162   
           15               note_on          137        30511      27.636800   
           16               note_on           12        30634      27.748213   

                            note  velocity  \
bar_number bar_beat_number                   
6          1                36.0      45.0   
           1                51.0      74.0   
  

# Pandas series and dataframe Indexing techniques

Working through some of these...
- https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
- https://github.com/ZaxR/pandas_multiindex_tutorial/blob/master/Pandas%20MultiIndex%20Tutorial.ipynb

In [28]:
dates = pd.date_range('1/1/2000', periods=24)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14', '2000-01-15', '2000-01-16',
               '2000-01-17', '2000-01-18', '2000-01-19', '2000-01-20',
               '2000-01-21', '2000-01-22', '2000-01-23', '2000-01-24'],
              dtype='datetime64[ns]', freq='D')

In [29]:
df = pd.DataFrame(np.random.randn(24,4),
                  index=dates, columns=['A','B', 'C','D'])
df

,A,B,C,D
2000-01-01,0.305962,-0.172249,0.033090,-2.694771
2000-01-02,-0.420785,1.962162,0.458413,-0.237421
2000-01-03,-0.162125,0.074268,1.004685,-0.097182
2000-01-04,0.272380,-0.451521,-0.677017,-1.191242
2000-01-05,1.275990,0.777484,-0.297850,0.678866
2000-01-06,-1.580413,0.563290,0.682548,1.609484
2000-01-07,-0.500073,-0.348740,1.798803,0.305215
2000-01-08,0.650310,-0.609270,1.600144,-0.684545
2000-01-09,0.447592,0.202940,0.024391,-0.174897
2000-01-10,-0.570527,1.567525,-0.010495,-0.235080


In [30]:
s = df['A']
my_i = 13
print('index {}: {}'.format(my_i, s[dates[my_i]]))
s[:my_i]

index 13: 0.5766598749574005


2000-01-01    0.305962
2000-01-02   -0.420785
2000-01-03   -0.162125
2000-01-04    0.272380
2000-01-05    1.275990
2000-01-06   -1.580413
2000-01-07   -0.500073
2000-01-08    0.650310
2000-01-09    0.447592
2000-01-10   -0.570527
2000-01-11    1.021143
2000-01-12    0.050225
2000-01-13   -1.498815
Freq: D, Name: A, dtype: float64

In [31]:
s[::3]  # select every 3rd element

2000-01-01    0.305962
2000-01-04    0.272380
2000-01-07   -0.500073
2000-01-10   -0.570527
2000-01-13   -1.498815
2000-01-16   -1.154869
2000-01-19    0.089200
2000-01-22    0.059398
Freq: 3D, Name: A, dtype: float64

In [32]:
# select rows 3 thru 8, not including 8, then
# columns 2 thru 3, not including 3
df.iloc[3:8, 2:3] 

# select rows 3 and 4, column 2
df.iloc[[3,4], [2]]

,C
2000-01-04,-0.677017
2000-01-05,-0.297850
2000-01-06,0.682548
2000-01-07,1.798803
2000-01-08,1.600144


,C
2000-01-04,-0.677017
2000-01-05,-0.297850


In [33]:
df.sample(3)  # select random sample

,A,B,C,D
2000-01-14,0.576660,0.044007,-0.824790,1.914514
2000-01-23,0.241446,0.378895,-1.680329,-0.875916
2000-01-19,0.089200,0.493572,0.659653,0.197540


## Boolean indexing series

This looks pretty powerful...
- https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing

In [34]:
s = pd.Series(range(-3,4))
s
s[s > 0]
s[(s < -2)|(s>1.8)]

0   -3
1   -2
2   -1
3    0
4    1
5    2
6    3
dtype: int64

4    1
5    2
6    3
dtype: int64

0   -3
5    2
6    3
dtype: int64

## MultiIndex and 'isin' stuff

using Series...

In [35]:
s_mi = pd.Series(np.arange(6),
                           index=pd.MultiIndex.from_product([[0,1], ['a','b','c']]))
s_mi

0  a    0
   b    1
   c    2
1  a    3
   b    4
   c    5
dtype: int64

In [36]:
# this matches on 2 levels of index
s_mi.iloc[s_mi.index.isin([(1,'a'), (0,'c')])]

0  c    2
1  a    3
dtype: int64

In [37]:
# specify index level to test against

# pull out everything with index 'c' at 
# level 1 (i.e. 0, 1, therefore 1 is 
# the second index level)
s_mi.iloc[s_mi.index.isin(['c'], level=1)]

0  c    2
1  c    5
dtype: int64

using DataFrames...

In [38]:
df = pd.DataFrame({'vals': [1,2,3,4], 'ids': ['a','b','f','n'],
                   'ids2': ['a','n','c','n']})
df

# search entire df for list of values
values = ['a','b',1,3]
df.isin(values)

,vals,ids,ids2
0,1,a,a
1,2,b,n
2,3,f,c
3,4,n,n


,vals,ids,ids2
0,True,True,True
1,False,True,False
2,True,False,False
3,False,False,False


In [39]:
# search only specific columns
values = {'ids': ['a','n'], 'vals': [1,4]}
df.isin(values)

,vals,ids,ids2
0,True,True,False
1,False,False,False
2,False,False,False
3,True,True,False


Renaming multiindexes, levels, etc. and whatnot ...

In [40]:
index = pd.MultiIndex.from_product([range(3),['wun','too']], names=['furst', 'secund'])
index

MultiIndex([(0, 'wun'),
            (0, 'too'),
            (1, 'wun'),
            (1, 'too'),
            (2, 'wun'),
            (2, 'too')],
           names=['furst', 'secund'])

In [41]:
print('look at second level index, at [1]: {}'.format(index.levels[1]))
index.set_levels(['x','y','t'], level=1, inplace=True)
index


look at second level index, at [1]: Index(['too', 'wun'], dtype='object', name='secund')


MultiIndex([(0, 'y'),
            (0, 'x'),
            (1, 'y'),
            (1, 'x'),
            (2, 'y'),
            (2, 'x')],
           names=['furst', 'secund'])

## Setting multi/index from columns, resetting

More info here ...
- https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#set-reset-index


# MultiIndex in depth

Digging in here, refs I found useful..
- https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html
